# Logistic Neurons

In [ ]:
import numpy as np
import jax
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from utils import make_classification, draw_decision_boundary, sigmoid

jnp = jax.numpy

plt.style.use('ggplot')
plt.rc('figure', figsize=(8, 6))

## Activation of a logistic neuron:

## $$ z = \sum_{i \in L} x_{i}w_{i} + b$$ 

## Predicted output:

## $$ h = \frac{1}{1 + e^{-z}} $$

## Log Loss:
## $$ E = -\sum_{i=1}^{n}\Big(y^i \log(h^{i}) + (1 - y^i) \log(1 - h^{i})\Big) $$

# Logistic Neuron in NumPy:

## Step 1: Make dummy data

In [ ]:
X, Y = make_classification()
W = np.random.rand(2, 1)
B = np.random.rand(1,)

plt.scatter(*X.T, c=Y.ravel())

In [ ]:
draw_decision_boundary(W.ravel().tolist() + [B[0]], X, Y.ravel())
plt.ylim(X[:, 1].min() - 0.1, X[:, 1].max() + 0.1)

## Step 2: Get activation and prediction

In [ ]:
# activation
Z = np.dot(X, W) + B

# prediction
Y_pred = sigmoid(Z)

## Step 3: Gradient Descent for Logistic Regression

## $$ \mathbf{W}_{t + 1} = \mathbf{W}_{t} - \alpha(\mathbf{X}^T(\sigma(\mathbf{XW}) - Y))$$

# Using this gradient to train neuron with NumPy

In [ ]:
def predict(X, weights, bias=None):
    z = np.dot(X, weights)
    if bias is not None:
        z += bias
    return sigmoid(z)

def train(X, Y, weights, alpha=0.001):
    y_hat = predict(X, weights)
    weights -= alpha * np.dot(X.T, y_hat - Y)
    return weights

def loss(y1, y2):
    return (0.5 * ((y1 - y2) ** 2)).sum()

In [ ]:
X, Y = make_classification()
W = np.random.rand(2, 1)
B = np.random.rand(1,)


for i in range(10000):
    y_hat = predict(X, W)
    W = train(X, Y, W)
    if i % 1000 == 0:
        print("Loss: ", loss(Y, y_hat))

In [ ]:
draw_decision_boundary(W.ravel().tolist() + [B[0]], X, Y.ravel())

# Logistic Regression with Jax

In [ ]:
X, Y = make_classification()
X, Y = map(jnp.array, (X, Y.ravel()))

# Initialize weights and biases
W = jnp.array(np.random.rand(2))
b = np.random.rand()

draw_decision_boundary(W.ravel().tolist() + [b], X, Y)
# plt.ylim(X[:, 1].min() - 0.1, X[:, 1].max() + 0.1)

In [ ]:
def sigmoid(x):
    return 1 / (1 + jnp.exp(-x))

def predict(X, W, b):
    return sigmoid(jnp.dot(X, W) + b)

def loss(W, b, X, Y):
    H = predict(X, W, b)
    l = Y * H + (1 - Y) * (1 - H)
    return -jnp.sum(jnp.log(l))

alpha = 1e-3
w_grad = jax.grad(loss, 0)
b_grad = jax.grad(loss, 1)

for i in range(10000):
    W -= alpha * w_grad(W, b, X, Y)
    b -= alpha * b_grad(W, b, X, Y)
    if i % 1000 == 0:
        print(loss(W, b, X, Y))

In [ ]:
draw_decision_boundary(W.tolist() + [b], X, Y)